In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 11.3MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
     |████████████████████████████████| 2.9MB 42.8MB/s 
     |████████████████████████████████| 1.1MB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=44961ad774d4d951df422181436473c564da10e8709e08939b7fe5f86a63b3b6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import logging
import os
import random
import json
import csv
import sys
from io import open
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import numpy as np
from google.colab import drive
from zipfile import ZipFile
import pandas as pd
import ast
from tqdm import tqdm
import re
import string


import torch
from torch.utils.data import (DataLoader,Dataset, random_split, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertPreTrainedModel
from transformers import BertModel
from transformers import WEIGHTS_NAME, BertConfig,BertForMaskedLM,BertTokenizer                                  
from transformers import AdamW, get_linear_schedule_with_warmup

#GPU Connection

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
drive.mount('/content/drive/')


Mounted at /content/drive/


#Tokenizer

In [ ]:
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic', do_lower_case=False)

Loading BERT tokenizer...


#Load Model

In [ ]:
model = BertForMaskedLM.from_pretrained(
    'asafaya/bert-base-arabic', 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

#Convert Dataset to Required Formatting

In [ ]:
class TextDataset(Dataset):
    """
    Used to turn .txt file into a suitable dataset object
    """

    def __init__(self, tokenizer, file_path, block_size=24):
        assert os.path.isfile(file_path)
        with open(file_path, encoding="utf-8") as f:
            text = f.read()
        lines = text.split("\n")
        self.examples = []
        for line in tqdm(lines):
            tokenized_text = tokenizer.encode(line, max_length=block_size,
                                              add_special_tokens=True, pad_to_max_length=True)  # Get ids from text
            self.examples.append(tokenized_text)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

In [ ]:
def create_dataset(tokenizer, file_path, block_size=24):
    """
    Creates a dataset object from file path.
    :param tokenizer: Bert tokenizer to create dataset
    :param file_path: Path where data is stored
    :param block_size: Should be in range of [0,512], viable choices are 64,
    128, 256, 512
    :return: The dataset
    """
    dataset = TextDataset(tokenizer, file_path=file_path,
                          block_size=block_size)
    return dataset

In [ ]:
dataset_train = create_dataset(tokenizer,"/content/drive/MyDrive/train_dataset1.txt",block_size=24)
dataset_test = create_dataset(tokenizer,"/content/drive/MyDrive/test_dataset1.txt",block_size=24)

  0%|          | 0/115696 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 12856/12856 [00:02<00:00, 4305.21it/s]


In [ ]:
def mask_tokens(inputs, tokenizer):
    """ Prepare masked tokens inputs/labels for masked language modeling:
    80% MASK, 10% random, 10% original.
    * The standard implementation from Huggingface Transformers library *
    """
    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training
    # (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    # MLM Prob is 0.15 in examples
    probability_matrix = torch.full(labels.shape, 0.15)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
        for val in
        labels.tolist()]
    probability_matrix.masked_fill_(torch.tensor(
        special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with
    # tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(
        labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(
        tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(
        labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(
        len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens
    # unchanged
    return inputs, labels

#Training

In [ ]:
batch_size=32
epochs=4
lr=5e-5
adam_epsilon=1e-8

In [ ]:
def accuracy(out, labels, total_test):
    class_preds = out.data.cpu().numpy().argmax(axis=-1)
    labels = labels.data.cpu().numpy()
    return np.sum(class_preds == labels) / total_test


   
def train(model, tokenizer, train_dataset, batch_size, lr, adam_epsilon,
          epochs):
    """
    :param model: Bert Model to train
    :param tokenizer: Bert Tokenizer to train
    :param train_dataset:
    :param batch_size: Stick to 1 if not using using a high end GPU
    :param lr: Suggested learning rate from paper is 5e-5
    :param adam_epsilon: Used for weight decay fixed suggested parameter is
    1e-8
    :param epochs: Usually a single pass through the entire dataset is
    satisfactory
    :return: Loss
    """

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=batch_size)

    t_total = len(train_dataloader) // batch_size  # Total Steps

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if
                    not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 0, t_total)

    # ToDo Case for fp16

    # Start of training loop
    print("***** Running training *****")
    print("  Num examples = ", len(train_dataset))
    print("  Batch size = ", batch_size)

    model.train()
    global_step = 0
    acc = 0
    total_test = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.resize_token_embeddings(len(tokenizer))
    model.zero_grad()
    train_iterator = trange(int(epochs), desc="Epoch")
    for _ in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for batch in epoch_iterator:
            inputs, labels = mask_tokens(batch, tokenizer)
            inputs = inputs.to('cuda')  # Don't bother if you don't have a gpu
            labels = labels.to('cuda')

            loss, pred_masks = model(inputs, masked_lm_labels=labels)
            # model outputs are always tuple in transformers (see doc)
            loss = loss
            loss.backward()
            tr_loss += loss.item()
            total_test += labels.nelement()
            acc += accuracy(pred_masks, labels, total_test)

            # if (step + 1) % 1 == 0: # 1 here is a placeholder for gradient
            # accumulation steps
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            global_step += 1

    print(" global_step = %s average loss = %s"%(global_step, tr_loss / global_step))
    print("Accuracy = " , 100 * acc)
    


    return model, tokenizer

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


model,tokenizer=train(model,tokenizer,dataset_train,batch_size,lr,adam_epsilon,epochs)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

***** Running training *****
  Num examples =  115696
  Batch size =  32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/transformers/modeling_bert.py:1152: FutureWarning: The `masked_lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,
Epoch:  25%|██▌       | 1/4 [25:15<1:15:47, 1515.84s/it]

Epoch:  50%|█████     | 2/4 [50:31<50:31, 1515.78s/it]  

Epoch:  75%|███████▌  | 3/4 [1:15:45<25:15, 1515.27s/it]

Epoch: 100%|██████████| 4/4 [1:41:00<00:00, 1515.14s/it]


 global_step = 14464 average loss = 1.66303923708362
Accuracy =  90.05310992072496


#Save Model

In [ ]:
output_dir = '/content/drive/MyDrive/model_save64k_asafaya'

In [ ]:
# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/model_save64k_asafaya


('/content/drive/MyDrive/model_save64k_asafaya/tokenizer_config.json',
 '/content/drive/MyDrive/model_save64k_asafaya/special_tokens_map.json',
 '/content/drive/MyDrive/model_save64k_asafaya/vocab.txt',
 '/content/drive/MyDrive/model_save64k_asafaya/added_tokens.json')

#Load Model After Training & Evaluate

In [ ]:
model = BertForMaskedLM.from_pretrained(
    output_dir, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model = model.cuda()

tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=False)

In [ ]:
def evaluate(model, tokenizer, eval_dataset, batch_size):
    """
    :param model: Newly trained Bert model
    :param tokenizer:Newly trained Bert tokenizer
    :param eval_dataset:
    :param batch_size: More flexible than training, the user can get away
    with picking a higher batch_size
    :return: The perplexity of the dataset
    """
    eval_sampler = SequentialSampler(eval_dataset)  # Same order samplinng
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=batch_size)

    # Eval!
    print("***** Running evaluation *****")
    print("  Num examples = %d", len(eval_dataset))
    print("  Batch size = %d", batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    acc = 0
    model.eval()
    total_test = 0
    # Evaluation loop
    for batch in tqdm_notebook(eval_dataloader, desc='Evaluating'):
        inputs, true_masks = mask_tokens(batch, tokenizer)
        inputs = inputs.to('cuda')
        true_masks = true_masks.to('cuda')
        with torch.no_grad():
            loss, pred_masks = model(inputs, masked_lm_labels=true_masks)

            lm_loss = loss
            eval_loss += lm_loss.mean().item()
  
            total_test += true_masks.nelement()
            acc += accuracy(pred_masks, true_masks, total_test)

        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss)).item()

    result = {
        'perplexity': "{:.3f}".format(perplexity),
        'eval_loss': "{:.3f}".format(eval_loss),
        'acc': "{:.3f}".format(100 * acc)
    }

    return result

In [ ]:
eval_loss = evaluate(model, tokenizer, dataset_test, batch_size)
print(eval_loss)

***** Running evaluation *****
  Num examples = %d 12856
  Batch size = %d 32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/transformers/modeling_bert.py:1152: FutureWarning: The `masked_lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,



{'perplexity': '3.259', 'eval_loss': '1.181', 'acc': '72.745'}
